In [1]:
import sympy as sp
import numpy as np
from IPython.display import display
from sympy.utilities.lambdify import lambdastr
import inspect
import matplotlib.pyplot as plt
from typing import List


In [2]:
class Code:
    def __init__(self, x_, f_):
        raw_code = inspect.getsource(
            sp.lambdify(x_, f_)).split('return')[-1]
        raw_code = raw_code.replace('exp', 'np.exp')
        raw_code = raw_code.replace('sqrt', 'np.sqrt')

        raw_code = raw_code.replace('sin', 'np.sin')
        raw_code = raw_code.replace('cos', 'np.cos')
        raw_code = raw_code.replace('tan', 'np.tan')

        raw_code = raw_code.replace('atan', 'np.arctan')
        raw_code = raw_code.replace('asin', 'np.arcsin')
        raw_code = raw_code.replace('acos', 'np.arccos')

        raw_code = raw_code.replace('tanh', 'np.tanh')
        raw_code = raw_code.replace('sinh', 'np.sinh')
        raw_code = raw_code.replace('cosh', 'np.cosh')
        self.raw_code = raw_code
        self.t = sp.lambdify(x_, f_)

In [3]:
class Error_propagation:
    def  __init__(self,func):
        self.func = func
        self.F = sp.Symbol('f')
        self.F_err = sp.Symbol('\Delta f')


        self.f = sp.parse_expr(self.func)
        print('input expression:')



        print('#'*20)
        print('INPUT')
        self._f = sp.latex(self.f)

        display(self.f)
        print('python rep:')
        display(Code(sp.Symbol('x'), self.f).raw_code.strip())
        print('#'*20)
        print('OUTPUT')
        self.substitution = False
        self.do_error()
        
    def k(self, symbol:str, f_:sp.Expr):
        a = sp.diff(f_, sp.Symbol(symbol))*sp.Symbol('\\Delta '+symbol)
        a = a**2
        return a

    def set_error_to_zero(self,params:List[str]):
        print('-'*20)
        p = self.error
        for param in params:
            var = '\\Delta '+param
            p = p.subs(var,0)

        print('substituted params\' error with 0: '+', '.join(params))


        self.error_0 = p
        display(sp.sqrt(self.error_0))
        self._err2 = sp.latex(self.error_0)

        err_simple = self.F*sp.simplify(sp.sqrt(self.error_0/self.f**2))
        display(err_simple)
        self._err2_simple = sp.latex(err_simple)

        self.substitution = True

    
    def do_error(self):

        self.error = sum(self.k(i, self.f) for i in list(map(str,self.f.free_symbols)))
        display(sp.sqrt(self.error))
        self._err1 = sp.latex(sp.sqrt(self.error))

        err_simple = self.F*sp.simplify(sp.sqrt(self.error/self.f**2))
        arg = sp.simplify(self.error/self.f**2)

        f = 0
        for summand in arg.args:
            for symbol in self.f.free_symbols:
                if symbol in summand.free_symbols and sp.Symbol('\\Delta '+str(symbol)) in summand.free_symbols:
                    f+=summand*(sp.Symbol('\\Delta '+str(symbol)+'_\\%')**2) * symbol**2/sp.Symbol('\\Delta '+str(symbol))**2
        err_rel = self.F*sp.simplify(sp.sqrt(f))
        display(err_rel)
        display(err_simple)
        self._err1_simple = sp.latex(err_simple)

    def print_errors(self):
        print(self._err1)
        print(self._err1_simple)
        if self.substitution:
            print(self._err2)
            print(self._err2_simple)

    def get_errors(self):
        return self._err1, self._err1_simple, self._err2, self._err2_simple


In [4]:
#E = Error_propagation('R_0*(1+(a*T)+(b*T**2)+((c*T**3)*(T-100)))')
#E.set_error_to_zero(['T','R_0'])
E = Error_propagation('5*cos(k)*b*c')
#E.set_error_to_zero(['a'])
#E.print_errors()

input expression:
####################
INPUT


5*b*c*cos(k)

python rep:


'(5*b*c*np.cos(k))'

####################
OUTPUT


sqrt(25*\Delta b**2*c**2*cos(k)**2 + 25*\Delta c**2*b**2*cos(k)**2 + 25*\Delta k**2*b**2*c**2*sin(k)**2)

f*sqrt(\Delta b_\%**2 + \Delta c_\%**2 + \Delta k_\%**2*k**2*tan(k)**2)

f*sqrt(\Delta b**2/b**2 + \Delta c**2/c**2 + \Delta k**2*tan(k)**2)

$\Delta f = \sqrt{\frac{4 \Delta a_{0}^{2} a_{0}^{2}}{\cos^{4}{\left(\frac{k_{1}}{k_{2}} \right)}} + \frac{4 \Delta k_{1}^{2} a_{0}^{4} \sin^{2}{\left(\frac{k_{1}}{k_{2}} \right)}}{k_{2}^{2} \cos^{6}{\left(\frac{k_{1}}{k_{2}} \right)}} + \frac{4 \Delta k_{2}^{2} a_{0}^{4} k_{1}^{2} \sin^{2}{\left(\frac{k_{1}}{k_{2}} \right)}}{k_{2}^{4} \cos^{6}{\left(\frac{k_{1}}{k_{2}} \right)}}} $

$\Delta fs = fs \sqrt{\frac{2 \Delta \theta^{2} \cos^{2}{\left(\frac{\theta}{2} \right)}}{1 - \cos{\left(\theta \right)}} + \frac{\Delta E_{2}^{2}}{E_{2}^{2}} + \frac{\Delta E_{1}^{2}}{E_{1}^{2}}}$

$\Delta fs\Big|_{\Delta E_1=0} = \sqrt{16 E_{1}^{2} E_{2}^{2} \Delta \theta^{2} \sin^{2}{\left(\frac{\theta}{2} \right)} \cos^{2}{\left(\frac{\theta}{2} \right)} + 16 E_{1}^{2} \Delta E_{2}^{2} \sin^{4}{\left(\frac{\theta}{2} \right)}}$\\
$\Delta fs\Big|_{\Delta E_1=0} = \sqrt{16 E_{1}^{2} E_{2}^{2} \Delta \theta^{2} \sin^{2}{\left(\frac{\theta}{2} \right)} \cos^{2}{\left(\frac{\theta}{2} \right)} + 16 E_{1}^{2} \Delta E_{2}^{2} \sin^{4}{\left(\frac{\theta}{2} \right)}}
\Delta fs\Big|_{\Delta E_1=0} = fs \sqrt{\frac{2 \Delta \theta^{2} \cos^{2}{\left(\frac{\theta}{2} \right)}}{1 - \cos{\left(\theta \right)}} + \frac{\Delta E_{2}^{2}}{E_{2}^{2}}}$\\
$$\\
$$\\